> **⚠️ EN COURS DE CONSTRUCTION**
>
> Ce notebook est en cours de finalisation. Merci de ne pas le consulter pour l'instant.

# Projet - Détecteur de Fake News

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 4h (2 sessions)  
**Prérequis** : TPs 1-4 (Fondamentaux NLP, Attention, Multi-Head, Transformer)

---

## Objectifs du projet

Dans ce projet, vous allez :

**Partie 1 - From Scratch (2h)**
1. Créer un tokenizer simple
2. Entraîner votre Transformer des TPs précédents
3. Évaluer les performances

**Partie 2 - Fine-tuning (2h)**
4. Utiliser un modèle pré-entraîné (DistilBERT)
5. Comparer les performances
6. Créer un pipeline multilingue (FR→EN→Classification)

---

# PARTIE 1 : Classification From Scratch

---

## 0. Installation et imports

In [ ]:
!pip install torch transformers datasets matplotlib numpy scikit-learn tqdm seaborn -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

torch.manual_seed(42)

## 1. Notre Transformer (des TPs précédents)

Nous réutilisons les composants développés dans les TPs 2, 3 et 4.

In [ ]:
# === Composants du Transformer (TPs 2-4) ===

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x, mask=None):
        B, S, _ = x.shape
        Q = self.W_q(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(B, S, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        out = (attn @ V).transpose(1, 2).contiguous().view(B, S, self.embed_dim)
        return self.W_o(out), attn


class FeedForward(nn.Module):
    def __init__(self, embed_dim, ff_dim=None, dropout=0.1):
        super().__init__()
        ff_dim = ff_dim or 4 * embed_dim
        self.net = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, embed_dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)


class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim=None, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.ff = FeedForward(embed_dim, ff_dim, dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        attn_out, _ = self.attn(self.norm1(x), mask)
        x = x + self.dropout(attn_out)
        x = x + self.dropout(self.ff(self.norm2(x)))
        return x


class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, num_classes,
                 max_len=512, dropout=0.1, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len, dropout)
        self.layers = nn.ModuleList([TransformerBlock(embed_dim, num_heads, dropout=dropout) 
                                     for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dim)
        self.classifier = nn.Linear(embed_dim, num_classes)
        self.embed_dim = embed_dim
    
    def forward(self, x, mask=None):
        x = self.embedding(x) * math.sqrt(self.embed_dim)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x, mask)
        x = self.norm(x)
        # Moyenne sur tous les tokens (alternative à [CLS])
        x = x.mean(dim=1)
        return self.classifier(x)

print("Transformer chargé !")

---

## 2. Chargement du Dataset

Nous utilisons le dataset LIAR, un dataset de fact-checking en anglais.

In [ ]:
from datasets import load_dataset

# Charger le dataset LIAR (fact-checking)
print("Chargement du dataset...")
dataset = load_dataset("liar", trust_remote_code=True)

print(f"\nStructure du dataset:")
print(dataset)

print(f"\nExemple:")
print(dataset['train'][0])

In [ ]:
# Le dataset LIAR a 6 labels, simplifions en 2 (fake vs real)
# Labels originaux: pants-fire, false, barely-true, half-true, mostly-true, true
# Fake: pants-fire, false, barely-true (0, 1, 2)
# Real: half-true, mostly-true, true (3, 4, 5)

def simplify_label(example):
    example['binary_label'] = 0 if example['label'] < 3 else 1
    return example

dataset = dataset.map(simplify_label)

# Vérifier la distribution
train_labels = [ex['binary_label'] for ex in dataset['train']]
print(f"Distribution train: Fake={train_labels.count(0)}, Real={train_labels.count(1)}")

---

## 3. Tokenization

On crée un tokenizer simple basé sur les mots (comme vu au TP1).

In [ ]:
# ============================================
# EXERCICE 1 : Créer un vocabulaire
# ============================================

class SimpleTokenizer:
    def __init__(self, texts, max_vocab_size=10000, min_freq=2):
        """
        Tokenizer simple basé sur les mots.
        """
        self.pad_token = '<PAD>'
        self.unk_token = '<UNK>'
        
        # Compter les mots
        word_counts = Counter()
        for text in texts:
            words = text.lower().split()
            word_counts.update(words)
        
        # TODO: Créer word2idx et idx2word
        # 1. Commencer par les tokens spéciaux (PAD=0, UNK=1)
        # 2. Ajouter les mots les plus fréquents (>= min_freq)
        # 3. Limiter à max_vocab_size
        
        self.word2idx = None  # À COMPLÉTER
        self.idx2word = None  # À COMPLÉTER
        self.vocab_size = None  # À COMPLÉTER
    
    def encode(self, text, max_len=128):
        """Convertit un texte en indices."""
        words = text.lower().split()[:max_len]
        indices = [self.word2idx.get(w, self.word2idx[self.unk_token]) for w in words]
        
        # Padding
        if len(indices) < max_len:
            indices += [self.word2idx[self.pad_token]] * (max_len - len(indices))
        
        return indices
    
    def decode(self, indices):
        """Convertit des indices en texte."""
        words = [self.idx2word.get(idx, self.unk_token) for idx in indices]
        return ' '.join(w for w in words if w != self.pad_token)

In [ ]:
# Créer le tokenizer
train_texts = [ex['statement'] for ex in dataset['train']]
tokenizer = SimpleTokenizer(train_texts, max_vocab_size=8000, min_freq=2)

# Test
test_text = "The president said the economy is doing great."
encoded = tokenizer.encode(test_text, max_len=20)
decoded = tokenizer.decode(encoded)

print(f"Original: {test_text}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

---

## 4. Dataset PyTorch

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['statement']
        label = item['binary_label']
        
        input_ids = self.tokenizer.encode(text, self.max_len)
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Créer les datasets
MAX_LEN = 64
BATCH_SIZE = 32

train_dataset = FakeNewsDataset(dataset['train'], tokenizer, MAX_LEN)
val_dataset = FakeNewsDataset(dataset['validation'], tokenizer, MAX_LEN)
test_dataset = FakeNewsDataset(dataset['test'], tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"Train: {len(train_dataset)} samples")
print(f"Val: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")

---

## 5. Entraînement

In [ ]:
# Créer le modèle
model_scratch = TransformerClassifier(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    num_heads=4,
    num_layers=2,
    num_classes=2,
    max_len=MAX_LEN,
    dropout=0.1,
    pad_idx=0
).to(device)

# Compter les paramètres
num_params = sum(p.numel() for p in model_scratch.parameters() if p.requires_grad)
print(f"Paramètres: {num_params:,}")

In [ ]:
# ============================================
# EXERCICE 2 : Boucle d'entraînement
# ============================================

def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in tqdm(loader, desc="Training", leave=False):
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)
        
        # TODO: Implémenter la boucle d'entraînement
        # 1. Zero grad
        # 2. Forward pass
        # 3. Calculer la loss
        # 4. Backward pass
        # 5. Optimizer step
        
        pass  # À COMPLÉTER
        
        # Stats (après avoir calculé outputs et loss)
        # total_loss += loss.item()
        # preds = outputs.argmax(dim=-1)
        # correct += (preds == labels).sum().item()
        # total += labels.size(0)
    
    return total_loss / len(loader), correct / total


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            preds = outputs.argmax(dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(loader), correct / total

In [ ]:
# Configuration
EPOCHS = 5
LR = 1e-4

optimizer = torch.optim.AdamW(model_scratch.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Historique
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("Début de l'entraînement...\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model_scratch, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model_scratch, val_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print()

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss')
axes[0].legend()

axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Val')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 6. Évaluation sur le Test Set

In [ ]:
# Évaluation finale
test_loss, test_acc = evaluate(model_scratch, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy (From Scratch): {test_acc:.4f}")

In [ ]:
# Rapport détaillé
model_scratch.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label']
        
        outputs = model_scratch(input_ids)
        preds = outputs.argmax(dim=-1).cpu()
        
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

print("Classification Report (From Scratch):")
print(classification_report(all_labels, all_preds, target_names=['Fake', 'Real']))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fake', 'Real'],
            yticklabels=['Fake', 'Real'])
plt.xlabel('Prédiction')
plt.ylabel('Vérité')
plt.title('Matrice de Confusion (From Scratch)')
plt.show()

---

# PARTIE 2 : Fine-tuning avec DistilBERT

---

Maintenant, comparons avec un modèle pré-entraîné.

## 7. Charger DistilBERT

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
# Charger tokenizer et modèle
model_name = "distilbert-base-uncased"

tokenizer_bert = AutoTokenizer.from_pretrained(model_name)
model_bert = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
).to(device)

num_params_bert = sum(p.numel() for p in model_bert.parameters())
print(f"Modèle chargé: {model_name}")
print(f"Paramètres: {num_params_bert:,}")
print(f"\nComparaison: From Scratch = {num_params:,} | DistilBERT = {num_params_bert:,}")
print(f"Ratio: DistilBERT est {num_params_bert/num_params:.0f}x plus grand")

In [ ]:
# Préparer les données pour HuggingFace
def tokenize_function(examples):
    return tokenizer_bert(
        examples['statement'], 
        padding='max_length', 
        truncation=True,
        max_length=128
    )

# Recharger le dataset pour éviter les conflits
dataset_hf = load_dataset("liar", trust_remote_code=True)
dataset_hf = dataset_hf.map(simplify_label)

# Tokenizer le dataset
tokenized_dataset = dataset_hf.map(tokenize_function, batched=True)

# Renommer la colonne label
tokenized_dataset = tokenized_dataset.rename_column('binary_label', 'labels')

# Garder seulement les colonnes nécessaires
tokenized_dataset = tokenized_dataset.remove_columns(
    [c for c in tokenized_dataset['train'].column_names 
     if c not in ['input_ids', 'attention_mask', 'labels']]
)

tokenized_dataset.set_format('torch')
print("Dataset tokenisé pour DistilBERT !")

## 8. Fine-tuning avec Trainer

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Métriques
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Trainer
trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
)

In [ ]:
# Entraîner
print("Entraînement de DistilBERT...")
trainer.train()

In [ ]:
# Évaluation
results = trainer.evaluate(tokenized_dataset['test'])
print(f"\nRésultats sur le test set:")
print(f"Accuracy (DistilBERT): {results['eval_accuracy']:.4f}")
print(f"\nComparaison:")
print(f"  From Scratch: {test_acc:.4f}")
print(f"  DistilBERT:   {results['eval_accuracy']:.4f}")

---

## 9. Pipeline multilingue (FR → EN → Classification)

In [ ]:
from transformers import pipeline

# Créer les pipelines
print("Chargement du traducteur...")
translator = pipeline("translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en")
classifier = pipeline(
    "text-classification", 
    model=model_bert, 
    tokenizer=tokenizer_bert, 
    device=0 if torch.cuda.is_available() else -1
)
print("Pipelines créés !")

In [ ]:
# ============================================
# EXERCICE 3 : Pipeline FR → Classification
# ============================================

class FakeNewsDetectorPro:
    """
    Détecteur de Fake News avancé avec support multilingue.
    """
    
    def __init__(self, classifier, translator=None):
        self.classifier = classifier
        self.translator = translator
        self.label_map = {'LABEL_0': 'FAKE', 'LABEL_1': 'REAL'}
    
    def predict(self, text, source_lang='en'):
        """
        Prédit si un texte est une fake news.
        
        Args:
            text: Texte à analyser
            source_lang: Langue du texte ('en' ou 'fr')
        
        Returns:
            dict avec 'label', 'confidence', 'translation'
        """
        # TODO: Implémenter
        # 1. Si source_lang == 'fr' et translator existe, traduire
        # 2. Classifier le texte (traduit ou original)
        # 3. Retourner un dict avec les résultats
        
        pass  # À COMPLÉTER

In [ ]:
# Créer le détecteur
detector = FakeNewsDetectorPro(classifier, translator)

# Tests en français
tests_fr = [
    "Le gouvernement a annoncé une réforme des retraites.",
    "Les vaccins contiennent des puces 5G pour nous contrôler.",
    "L'économie française a progressé de 0.5% ce trimestre.",
    "Des pyramides ont été découvertes sur Mars par la NASA.",
]

print("=" * 70)
print("DÉTECTION DE FAKE NEWS (Français)")
print("=" * 70)

for text in tests_fr:
    result = detector.predict(text, source_lang='fr')
    if result:  # Si l'exercice est complété
        emoji = "❌" if result['label'] == 'FAKE' else "✅"
        print(f"\n{emoji} {result['label']} ({result['confidence']:.1%})")
        print(f"   FR: {text}")
        if result.get('translation'):
            print(f"   EN: {result['translation']}")

---

## 10. Analyse comparative

Comparons les deux approches.

In [ ]:
# ============================================
# EXERCICE 4 : Tableau comparatif
# ============================================

print("=" * 60)
print("COMPARAISON DES APPROCHES")
print("=" * 60)

print(f"\n{'Métrique':<25} {'From Scratch':<15} {'DistilBERT':<15}")
print("-" * 55)

# TODO: Compléter le tableau avec vos résultats
print(f"{'Paramètres':<25} {num_params:,}")
print(f"{'Test Accuracy':<25} {test_acc:.4f}")
print(f"{'Epochs':<25} 5")

# Questions de réflexion :
# 1. Pourquoi DistilBERT obtient-il de meilleurs résultats ?
# 2. Quels sont les avantages de l'approche From Scratch ?
# 3. Dans quel contexte utiliseriez-vous chaque approche ?

---

## 11. Analyse des erreurs

In [ ]:
# Analyser les erreurs sur un échantillon
test_data = dataset['test']

sample_size = 100
indices = np.random.choice(len(test_data), sample_size, replace=False)

errors = []

for idx in tqdm(indices, desc="Analyse"):
    item = test_data[int(idx)]
    text = item['statement']
    true_label = item['binary_label']
    
    result = classifier(text)[0]
    pred_label = 0 if result['label'] == 'LABEL_0' else 1
    
    if pred_label != true_label:
        errors.append({
            'text': text,
            'true': 'FAKE' if true_label == 0 else 'REAL',
            'pred': 'FAKE' if pred_label == 0 else 'REAL',
            'confidence': result['score']
        })

print(f"\nErreurs trouvées: {len(errors)}/{sample_size}")

In [ ]:
# Examiner quelques erreurs
print("\n" + "="*70)
print("EXEMPLES D'ERREURS")
print("="*70)

for i, err in enumerate(errors[:5]):
    print(f"\n[{i+1}] {err['text'][:80]}...")
    print(f"    Vérité: {err['true']} | Prédiction: {err['pred']} ({err['confidence']:.0%})")

---

## 12. Conclusion

### Ce que vous avez appris

**Partie 1 - From Scratch :**
- Créer un tokenizer simple
- Réutiliser les composants des TPs précédents
- Implémenter une boucle d'entraînement complète

**Partie 2 - Fine-tuning :**
- Utiliser des modèles pré-entraînés (HuggingFace)
- Fine-tuner avec le Trainer
- Créer un pipeline multilingue

### Comparaison

| Aspect | From Scratch | Fine-tuning |
|--------|-------------|-------------|
| Compréhension | ✅ Totale | ⚠️ Boîte noire |
| Performance | ⚠️ Limitée | ✅ État de l'art |
| Temps dev | ⚠️ Long | ✅ Rapide |
| Données requises | ⚠️ Beaucoup | ✅ Peu |

### Limitations

- Dataset anglais → Biais culturels
- Détecte des **patterns**, pas des **faits**
- Traduction peut introduire des erreurs

### Pour aller plus loin

- Modèles multilingues (mBERT, XLM-RoBERTa)
- Intégration de sources de fact-checking
- Analyse de la source et du contexte